In [ ]:
from __future__ import print_function

import os
import time

import ipywidgets as widgets
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import yaml
from IPython.display import display
from ipywidgets import (AppLayout, fixed, interact, interact_manual,
                        interactive, interactive_output)

from utils import data_importer, fcrn, metrics
from utils import model as md
from utils import unet
from utils import visualizer as vis

# Log Visualizations

In [ ]:
metric_list = ['MAPE', 'MSPE', 'RMSE', 'RMSELog', 'Log10', 'delta_threshold_1', 'delta_threshold_2', 'delta_threshold_3', 'SILog']
dashboard_list = ['Metrics Dashboard', 'Samples Dashboard']



def dashboard_sellector(dashboard_type, log_dir, table_dir):
    if dashboard_type == 'Metrics Dashboard':
        vis.MetricsDashboard(log_dir, metric_list)()
    if dashboard_type == 'Samples Dashboard':
        vis.SamplesDashboard(log_dir, table_dir)()
    
        
dashboard_type = widgets.Dropdown(
    options=dashboard_list,
    description='Dashboard Type:',
    style = {'description_width': 'initial'}
    )

log_dir = widgets.Text(
    value='./log/SynthiaSf_depth',
    placeholder='Insert log direction',
    description='Log Direction:',
                    )

table_dir = widgets.Text(
    value='./utils/labels.csv',
    placeholder='Insert table direction',
    description='Lable-table Direction:',
                    )

general_widgets = widgets.HBox([dashboard_type, log_dir, table_dir],
                              layout = {'border': '1px solid white', 'justify_content': 'space-between'})
output = interactive_output(dashboard_sellector, {'dashboard_type': dashboard_type, 'log_dir': log_dir, 'table_dir':table_dir})
display(general_widgets, output)

# Plotly

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
import plotly

In [ ]:
deptmap = label_list[0][0][:,:,0]

In [ ]:
x_l = []
y_l = []
z_l = []
c_l = []
s_l = []
for x in range(deptmap.shape[0]):
    for y in range(deptmap.shape[1]):
        x_l.append(x)
        y_l.append(y)
        z_l.append(deptmap[x,y]*1000)
        c_l.append(deptmap[x,y]*1000)
        s_l.append(0.1)
df = pd.DataFrame({'x':x_l, 'y':y_l, 'z':z_l,  'color':c_l, 'size':s_l})

In [ ]:
fig = px.scatter(df, x='y', y='x',color='color', size='size', hover_name='color',  render_mode='webgl')

In [ ]:
# iris = px.data.iris()
fig = px.scatter_3d(df, x='x', y='y', z='z', color='color', size='size')
# fig = px.scatter(df, x='y', y='x',color='color', size='size', hover_name='color',  render_mode='webgl')
fig.data[0].marker.line.width=0
fig.data[0].marker.size = 1
fig.layout.yaxis.autorange='reversed'
fig.show()

In [ ]:
plotly.io.write_html(fig, file="m.html")

# Dataset And Model' Output Histogram

In [ ]:
COMMON_SETTING = {
    'table_address' : "./utils/labels.csv",
    'batch_size' : 1, 
    'output_shape' : (480, 640),
    'data_type' : 'float32',
    'feature_types' : ['image', 'segmentation' ],
    'label_types' : ['depth'],# ['depth'], #'semantic_depth'
    'class_ids' : 'all'#[55,]#data_importer.COMMON_LABEL_IDS.copy()#
}

############################ SYNTHIA_SF_SETTINGS ########################################
address = 'C:/Users/KINEMATIX/Amin/Datasets/SYNTHIA-SF'#'D:/Library/Datasets/SYNTHIA-SF'#
dsg = data_importer.SynthiaSf
usage_range_train = (0, 1)
usage_range_validation = (0.75, 1)
usage_range_test = (0.75, 1)

dsg_train = dsg(**COMMON_SETTING,
                dataset_dir=address,
                usage='train',
                usage_range=usage_range_train)

dsg_validation = dsg(**COMMON_SETTING,
                    dataset_dir=address,
                    usage='validation',
                    usage_range=usage_range_validation)

In [ ]:
dsg_train.index = dsg_train.start_index
preview = iter(dsg_train)
size = int(dsg_train.size/dsg_train.batch_size)

try:
    del(train_set_hist)
except NameError:
    pass

nn_mean = 0
n_mean  = 0

for i in range(size):
    feature_list, label_list=next(preview)
    depth_data = label_list[0][0, :, :, 0]
    hist = np.histogram(depth_data, np.arange(0,1050,50))[0]
    try:
        train_set_hist = hist + train_set_hist
    except NameError:
        train_set_hist = hist
        
    nn_mean_error +=  abs(result[0, :, :, 0] - depth_data).mean()/size
    n_mean_error  += abs(result[0, :, :, 0] - depth_data/1000).mean()/size
    print(i/size*100)
        
train_set_hist_bins = np.histogram(depth_data, 20)[1]

In [ ]:
num_of_classes = dsg_train.class_ids.size
model = fcrn.model(img_shape=(480,640,3))
model.compile(optimizer="adam", loss=metrics.MAPE())

In [ ]:
preview = iter(dsg_train)
feature_list, label_list=next(preview)
result = model.predict(feature_list)

#### Normalized

In [ ]:
predict_hist = np.histogram(result, bins=np.arange(0,3,0.05))

In [ ]:
depth_label = ['  Depth Labels' for _ in range(59)]
d = np.append(train_set_hist, np.zeros(39))
data1 = pd.DataFrame({'depth': predict_hist[1][1:], 'percent': d*100/d.sum(), 'label': depth_label})

depth_label = ['  Prediction' for _ in range(59)]
data2 = pd.DataFrame({'depth': predict_hist[1][1:], 'percent': predict_hist[0]*100/predict_hist[0].sum(), 'label': depth_label})

a = data2.append(data1)
a.index = np.arange(118)

In [ ]:
fig = px.bar(a, x='depth', y='percent',  log_y=True, height=500, width=800,  color='label',barmode='overlay', labels={'label':""}, 
       title="Comparison of Untrained FCRN Model's Output and Normalized Depth Maps in Histogram")
fig.write_html("Comparison of Untrained FCRN Model's Output and Normalized Depth Maps in Histogram.html")
fig

In [ ]:
predict_mean = (predict_hist[0]*predict_hist[1][1:]).sum()/predict_hist[0].sum()

In [ ]:
depth_maps_mean = (d*predict_hist[1][1:]).sum()/d.sum()

In [ ]:
abs(predict_mean - depth_maps_mean) / n_mean_error

In [ ]:
n_mean_error

#### Non-normalized

In [ ]:
predict_hist = np.histogram(result, bins=train_set_hist_bins)

In [ ]:
depth_label = ['  Depth Labels' for _ in range(20)]
data1 = pd.DataFrame({'depth': train_set_hist_bins[1:], 'percent': train_set_hist*100/train_set_hist.sum(), 'label': depth_label})

depth_label = ['  Prediction' for _ in range(20)]
data2 = pd.DataFrame({'depth': train_set_hist_bins[1:], 'percent': predict_hist[0]*100/predict_hist[0].sum(), 'label': depth_label})

a = data2.append(data1)
a.index = np.arange(40)

In [ ]:
fig = px.bar(a, x='depth', y='percent',  log_y=True, height=500, width=800,  color='label',barmode='overlay', labels={'label':""}, 
       title="Comparison of Untrained FCRN Model's Output and Non-normalized Depth Maps in Histogram")
fig.write_html("Comparison of Untrained FCRN Model's Output and Non-normalized Depth Maps in Histogram.html")
fig

In [ ]:
predict_mean = (predict_hist[0]*predict_hist[1][1:]).sum()/predict_hist[0].sum()

In [ ]:
depth_maps_mean = (train_set_hist*predict_hist[1][1:]).sum()/train_set_hist.sum()

In [ ]:
abs(predict_mean - depth_maps_mean) / nn_mean_error

In [ ]:
nn_mean_error